# Converting SPI Values to Admin 1 and Admin 2

In [16]:
%load_ext jupyter_black
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [41]:
data_dir = Path(os.getenv("HAM_DIR")) / "precipitation/spi/"
output_dir = Path(os.getenv("HAM_DIR")) / "precipitation/"
shp_dir = Path(os.getenv("AA_DATA_DIR")) / "public/raw/ner/cod_ab/ner_cod_ab/"

In [44]:
ner_shp_adm0 = gpd.read_file(shp_dir / "adm0.shp.zip")
ner_shp_adm1 = gpd.read_file(shp_dir / "adm1.shp.zip")
ner_shp_adm2 = gpd.read_file(shp_dir / "adm2.shp.zip")

In [23]:
ner_shp_adm0.total_bounds

array([ 0.16625  , 11.69697  , 15.99564  , 23.5331862])

## Admin 1

In [24]:
ner_shp_adm1.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "ISO3", "ISO2"], axis=1, inplace=True
)
ner_shp_adm1

,adm_01,rowcacode1,NOMREG,geometry
0,Agadez,NER001,AGADEZ,"POLYGON ((11.98901 23.53319, 11.99796 23.51602..."
1,Diffa,NER002,DIFFA,"POLYGON ((15.55942 18.00625, 15.55675 17.95649..."
2,Dosso,NER003,DOSSO,"POLYGON ((3.66840 11.98401, 3.66388 11.98413, ..."
3,Maradi,NER004,MARADI,"POLYGON ((7.76947 14.42108, 7.82971 14.41669, ..."
4,Niamey,NER008,NIAMEY,"POLYGON ((2.09448 13.61969, 2.09711 13.61829, ..."
5,Tahoua,NER005,TAHOUA,"POLYGON ((4.87129 18.67932, 4.83798 18.58273, ..."
6,Tillabéri,NER006,TILLABERI,"POLYGON ((3.88129 15.68909, 4.01208 15.39410, ..."
7,Zinder,NER007,ZINDER,"POLYGON ((12.00488 17.41852, 12.00427 15.99390..."


In [36]:
file_list = os.listdir(data_dir)

adm1_spi_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm1,
        array,
        stats=["mean", "median"],
        nodata=29999,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm1_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm1[["rowcacode1", "NOMREG"]],
        adm1_stats,
        left_index=True,
        right_index=True,
    )
    file_df["date"] = file[(file.find("data") + 4) : file.find(".tiff")]
    adm1_spi_df = pd.concat([adm1_spi_df, file_df], axis=0)

In [37]:
adm1_spi_df

,rowcacode1,NOMREG,mean,median,date
0,NER001,AGADEZ,169.109286,174.0,202209
1,NER002,DIFFA,138.441888,146.0,202209
2,NER003,DOSSO,96.784788,97.0,202209
3,NER004,MARADI,135.712377,139.5,202209
4,NER008,NIAMEY,69.703704,77.0,202209
...,...,...,...,...,...
3,NER004,MARADI,186.933896,208.0,198101
4,NER008,NIAMEY,193.074074,195.0,198101
5,NER005,TAHOUA,208.813042,252.0,198101
6,NER006,TILLABERI,209.973156,208.0,198101


In [42]:
adm1_spi_df.to_csv(output_dir / "ner_adm1_spi.csv", index=False)

## Admin 2

In [45]:
ner_shp_adm2.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "adm_01", "rowcacode1", "ISO3", "ISO2"],
    axis=1,
    inplace=True,
)
ner_shp_adm2

,adm_02,rowcacode2,NOMDEP,geometry
0,Abala,NER006001,ABALA,"POLYGON ((3.88129 15.68909, 4.01208 15.39410, ..."
1,Abalak,NER005001,ABALAK,"POLYGON ((6.60449 16.28528, 6.67108 16.12830, ..."
2,Aderbissinat,NER001001,ADERBISSINAT,"POLYGON ((10.98339 17.64841, 10.99670 17.10229..."
3,Aguié,NER004001,AGUIE,"POLYGON ((7.78027 13.70728, 7.79572 13.69031, ..."
4,Arlit,NER001002,ARLIT,"POLYGON ((6.39447 17.90632, 6.19025 17.90357, ..."
...,...,...,...,...
62,Tillabéri,NER006012,TILLABERI,"POLYGON ((1.55408 14.64893, 1.57910 14.61090, ..."
63,Tillia,NER005012,TILLIA,"POLYGON ((4.98010 16.32788, 5.04633 16.29150, ..."
64,Torodi,NER006013,TORODI,"POLYGON ((1.50208 13.57770, 1.51208 13.56927, ..."
65,Ville de Tahoua,NER005013,VILLE DE TAHOUA,"POLYGON ((5.24548 15.00671, 5.24688 15.00409, ..."


In [47]:
adm2_spi_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm2,
        array,
        stats=["mean", "median"],
        nodata=29999,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm2_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm2[["rowcacode2", "NOMDEP"]],
        adm2_stats,
        left_index=True,
        right_index=True,
    )
    file_df["date"] = file[(file.find("data") + 4) : file.find(".tiff")]
    adm2_spi_df = pd.concat([adm2_spi_df, file_df], axis=0)

In [48]:
adm2_spi_df

,rowcacode2,NOMDEP,mean,median,date
0,NER006001,ABALA,74.981250,66.0,202209
1,NER005001,ABALAK,101.697588,99.0,202209
2,NER001001,ADERBISSINAT,174.019723,176.0,202209
3,NER004001,AGUIE,141.172840,155.0,202209
4,NER001002,ARLIT,137.101611,142.0,202209
...,...,...,...,...,...
62,NER006012,TILLABERI,219.122727,208.0,198101
63,NER005012,TILLIA,216.453313,252.0,198101
64,NER006013,TORODI,176.171533,195.0,198101
65,NER005013,VILLE DE TAHOUA,248.700000,252.0,198101


In [49]:
adm2_spi_df.to_csv(output_dir / "ner_adm2_spi.csv", index=False)